In [76]:
import pandas as pd 
from datetime import datetime
from forex_python.converter import CurrencyRates
import re


In [111]:
Expedia_hotels = pd.read_csv('Expedia_hotels.csv')

C:\Users\yarin\AppData\Local\Temp\ipykernel_5148\1586826330.py:1: DtypeWarning: Columns (13,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  Expedia_hotels = pd.read_csv('Expedia_hotels.csv')


In [112]:
df = Expedia_hotels.copy()

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Snapshot                36000 non-null  object 
 1   Index                   36000 non-null  int64  
 2   Hotel Name              35980 non-null  object 
 3   TTT                     35980 non-null  float64
 4   LOS                     35980 non-null  float64
 5   Grade                   35468 non-null  float64
 6   Num of Reviews          35468 non-null  object 
 7   Curr Price              35980 non-null  object 
 8   Original Price          15342 non-null  object 
 9   Percentage of discount  35980 non-null  float64
 10  Distance from center    35980 non-null  float64
 11  Type of room            35980 non-null  float64
 12  Location grade          35980 non-null  float64
 13  Is refundable           35980 non-null  object 
 14  Late payment            35980 non-null

In [114]:
df['Snapshot'] = df['Snapshot'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f').date())
df = df.loc[df['Index'] < 100]
df['Index'] = df['Index'] + 1
df['TTT'] = df['TTT'].astype('int64')
df['LOS'] = df['LOS'].astype('int64')


In [115]:
df['Num of Reviews'] = df['Num of Reviews'].apply(lambda x: int(re.sub("[^0-9]", "", x)) if not pd.isnull(x) else x)

In [116]:
df['Num of Reviews']

0        1296.0
1        1002.0
2         305.0
3        3123.0
4        1465.0
          ...  
35975    1000.0
35976    1000.0
35977    1001.0
35978     999.0
35979    1106.0
Name: Num of Reviews, Length: 30000, dtype: float64

In [117]:
df['Grade'].describe()

count    29494.000000
mean         8.404733
std          0.645410
min          5.600000
25%          8.000000
50%          8.600000
75%          8.800000
max         10.000000
Name: Grade, dtype: float64

In [118]:
df.loc[pd.isnull(df['Grade'])]['Hotel Name'].value_counts()

Latisha luxury place                148
Moxy Brooklyn Willamsburg           115
The FIDI Hotel                      111
The Ritz-Carlton New York, NoMad    107
The Flat NYC                         23
Latisha house                         2
Name: Hotel Name, dtype: int64

In [119]:
df.loc[pd.isnull(df['Num of Reviews'])]['Hotel Name'].value_counts()

Latisha luxury place                148
Moxy Brooklyn Willamsburg           115
The FIDI Hotel                      111
The Ritz-Carlton New York, NoMad    107
The Flat NYC                         23
Latisha house                         2
Name: Hotel Name, dtype: int64

In [120]:
df['Curr Price'] = df['Curr Price'].apply(lambda x: x.split('€')[1])

In [121]:
df['Original Price'] = df['Original Price'].apply(lambda x: x.split('€')[1] if not pd.isnull(x) else x)

In [122]:
df['Original Price']

0          360
1          NaN
2          493
3          NaN
4          NaN
         ...  
35975      NaN
35976    1,958
35977    2,150
35978      NaN
35979      NaN
Name: Original Price, Length: 30000, dtype: object

In [123]:
df['Original Price'] = df.apply(lambda x: x['Curr Price']  if pd.isnull(x['Original Price']) else x['Original Price'], axis = 1)
df['Original Price']

0          360
1          217
2          493
3          328
4          428
         ...  
35975    1,656
35976    1,958
35977    2,150
35978    1,926
35979    1,461
Name: Original Price, Length: 30000, dtype: object

In [124]:
df['Original Price'] = df['Original Price'].apply(lambda x: int(x.replace(',','')))
df['Curr Price'] = df['Curr Price'].apply(lambda x: int(x.replace(',','')))

In [125]:
df['Original Price']  

0         360
1         217
2         493
3         328
4         428
         ... 
35975    1656
35976    1958
35977    2150
35978    1926
35979    1461
Name: Original Price, Length: 30000, dtype: int64

In [126]:
c = CurrencyRates()
x = df['Snapshot'].unique()
curr_dict = {}
for date in x:
    curr_dict[date] = c.get_rate('EUR','USD', date) 



In [127]:
curr_dict[df['Snapshot'].iloc[0]]

1.0646

In [128]:
df['Original Price']

0         360
1         217
2         493
3         328
4         428
         ... 
35975    1656
35976    1958
35977    2150
35978    1926
35979    1461
Name: Original Price, Length: 30000, dtype: int64

In [129]:
df['Original Price'] = df.apply(lambda x: (x['Original Price']*curr_dict[x['Snapshot']])/x['LOS']  , axis = 1)
df['Curr Price'] = df.apply(lambda x: (x['Curr Price']*curr_dict[x['Snapshot']])/x['LOS']  , axis = 1)


In [130]:
df['Original Price'] = df['Original Price'].apply(lambda x: round(x,1))
df['Curr Price'] = df['Curr Price'].apply(lambda x: round(x,1))

In [131]:
df[['Original Price','Curr Price']]

,Original Price,Curr Price
0,383.3,296.0
1,231.0,231.0
2,524.8,478.0
3,349.2,349.2
4,455.6,455.6
...,...,...
35975,353.2,353.2
35976,417.6,351.3
35977,458.6,412.9
35978,410.8,410.8


In [132]:
df.drop(['Percentage of discount', 'Distance from center', 'Type of room',
       'Location grade'], axis = 1, inplace=True)

In [133]:
df[['Is refundable','Late payment', 'Option Member']] = df[['Is refundable','Late payment', 'Option Member']].astype('int64')

In [137]:
df['Extras included'] = df['Extras included'].apply(lambda x: 1 if not pd.isnull(x) else 0)

In [140]:
df.head(20)

,Snapshot,Index,Hotel Name,TTT,LOS,Grade,Num of Reviews,Curr Price,Original Price,Is refundable,Late payment,Extras included,Option Member
0,2023-03-06,1,Hotel 32 32,1,1,8.0,1296.0,296.0,383.3,0,0,0,0
1,2023-03-06,2,Sonesta Simply Suites Jersey City,1,1,8.4,1002.0,231.0,231.0,1,0,0,0
2,2023-03-06,3,Park Lane New York,1,1,8.8,305.0,478.0,524.8,0,0,0,0
3,2023-03-06,4,Arlo Midtown,1,1,9.2,3123.0,349.2,349.2,1,0,0,1
4,2023-03-06,5,"PUBLIC, an Ian Schrager hotel",1,1,8.0,1465.0,455.6,455.6,0,0,0,1
5,2023-03-06,6,Hard Rock Hotel New York,1,1,9.2,716.0,577.0,577.0,1,0,0,1
6,2023-03-06,7,Hotel Edison Times Square,1,1,7.8,2758.0,272.5,272.5,1,0,1,1
7,2023-03-06,8,YOTEL New York Times Square,1,1,7.0,1889.0,234.2,255.5,0,0,0,1
8,2023-03-06,9,Moxy Brooklyn Willamsburg,1,1,NaN,NaN,257.6,257.6,0,0,0,0
9,2023-03-06,10,New York Marriott Marquis,1,1,9.0,1543.0,572.8,572.8,0,0,0,0
